# Retrain a VGG16 Architecture
* https://keras.io/applications/#vgg16
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [7]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.8


In [8]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [9]:
# the larger the longer it takes, be sure to also adapt input layer size auf vgg network to this value

INPUT_SHAPE = (64, 64)
# INPUT_SHAPE = (128, 128)
# INPUT_SHAPE = (256, 256)

In [10]:
EPOCHS = 50

In [11]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 100

In [12]:
!rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [13]:
!ls -lh

total 317M
-rw-rw-r-- 1 ubuntu ubuntu  44K Oct  1 08:04 440px-Beagle_Upsy.jpg
drwxrwxr-x 8 ubuntu ubuntu 4.0K Oct  1 08:10 augmented-signs
-rw-rw-r-- 1 ubuntu ubuntu  17M Oct  1 08:10 augmented-signs.zip
-rw-rw-r-- 1 ubuntu ubuntu 303K Sep 27 15:22 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rw-rw-r-- 1 ubuntu ubuntu 844K Oct  1 08:04 cat-bonkers.png
-rw-rw-r-- 1 ubuntu ubuntu 140K Sep 27 15:22 cnn-augmentation.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.6M Oct  1 08:04 cnn-comparing-all-models.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 484K Oct  1 09:57 cnn-imagenet-retrain.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 158K Oct  1 08:04 cnn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.3M Oct  1 08:04 cnn-prediction.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 158K Oct  1 08:04 cnn-standard-architectures.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 198K Oct  1 08:04 cnn-train-augmented.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 495K Sep 27 15:22 london.jpg
drwxrwxr-x 3 ubuntu ubuntu 4.0K Sep 27 15:25 __MACOSX
-rw-rw-r-- 1 ubuntu ubuntu 127K Se

In [14]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [15]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
original_images, original_labels = load_data(original_dir, type=".ppm")

In [16]:
X, y = original_images, original_labels

### Uncomment next three cells if you want to train on augmented image set
#### Otherwise Overfitting can not be avoided because image set is simply too small

In [17]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip
# from zipfile import ZipFile
# zip = ZipFile('augmented-signs.zip')
# zip.extractall('.')

In [18]:
data_dir = os.path.join(ROOT_PATH, "augmented-signs")
augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [19]:
# merge both data sets

all_images = np.vstack((X, augmented_images))
all_labels = np.vstack((y, augmented_labels))

# shuffle
# https://stackoverflow.com/a/4602224

p = numpy.random.permutation(len(all_labels))
shuffled_images = all_images[p]
shuffled_labels = all_labels[p]
X, y = shuffled_images, shuffled_labels

### Split test and train data 80% to 20%

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((3335, 64, 64, 3), (3335, 6))

## First Step: Load VGG pretrained on imagenet and remove classifier
### Hope: Feature Extraction will also work well for Speed Limit Signs

![VGG architecture](https://djcordhose.github.io/ai/img/sketch/vgg-no-classifier.png)

### Imagenet
* Collection of labelled images from many categories
* http://image-net.org/

http://image-net.org/about-stats

<table class="table-stats" style="width: 500px">
<tbody><tr>
<td width="25%"><b>High level category</b></td>
<td width="20%"><b># synset (subcategories)</b></td>
<td width="30%"><b>Avg # images per synset</b></td>
<td width="25%"><b>Total # images</b></td>
</tr>

<tr><td>amphibian</td><td>94</td><td>591</td><td>56K</td></tr>

<tr><td>animal</td><td>3822</td><td>732</td><td>2799K</td></tr>

<tr><td>appliance</td><td>51</td><td>1164</td><td>59K</td></tr>

<tr><td>bird</td><td>856</td><td>949</td><td>812K</td></tr>

<tr><td>covering</td><td>946</td><td>819</td><td>774K</td></tr>

<tr><td>device</td><td>2385</td><td>675</td><td>1610K</td></tr>

<tr><td>fabric</td><td>262</td><td>690</td><td>181K</td></tr>

<tr><td>fish</td><td>566</td><td>494</td><td>280K</td></tr>

<tr><td>flower</td><td>462</td><td>735</td><td>339K</td></tr>

<tr><td>food</td><td>1495</td><td>670</td><td>1001K</td></tr>

<tr><td>fruit</td><td>309</td><td>607</td><td>188K</td></tr>

<tr><td>fungus</td><td>303</td><td>453</td><td>137K</td></tr>

<tr><td>furniture</td><td>187</td><td>1043</td><td>195K</td></tr>

<tr><td>geological formation</td><td>151</td><td>838</td><td>127K</td></tr>

<tr><td>invertebrate</td><td>728</td><td>573</td><td>417K</td></tr>

<tr><td>mammal</td><td>1138</td><td>821</td><td>934K</td></tr>

<tr><td>musical instrument</td><td>157</td><td>891</td><td>140K</td></tr>


<tr><td>plant</td><td>1666</td><td>600</td><td>999K</td></tr>

<tr><td>reptile</td><td>268</td><td>707</td><td>190K</td></tr>

<tr><td>sport</td><td>166</td><td>1207</td><td>200K</td></tr>

<tr><td>structure</td><td>1239</td><td>763</td><td>946K</td></tr>

<tr><td>tool</td><td>316</td><td>551</td><td>174K</td></tr>

<tr><td>tree</td><td>993</td><td>568</td><td>564K</td></tr>

<tr><td>utensil</td><td>86</td><td>912</td><td>78K</td></tr>

<tr><td>vegetable</td><td>176</td><td>764</td><td>135K</td></tr>

<tr><td>vehicle</td><td>481</td><td>778</td><td>374K</td></tr>

<tr><td>person</td><td>2035</td><td>468</td><td>952K</td></tr>

</tbody></table>

### Might be more suitable for cats and dogs, but is the best we have right now

In [21]:
from keras import applications
# applications.VGG16?
vgg_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
# vgg_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
# vgg_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

### All Convolutional Blocks are kept fully trained, we just removed the classifier part

In [22]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

### Next step is to push all our signs through the net just once and record the output of bottleneck features
#### Don't get confused: this is no training, yet, this just is recording the prediction in order not to repeat this expensive step over and over again when we train the classifier later

In [23]:
# will take a while, but not really long depending on size and number of input images

%time bottleneck_features_train = vgg_model.predict(X_train)

CPU times: user 7.03 s, sys: 1.34 s, total: 8.36 s
Wall time: 7.86 s


In [24]:
bottleneck_features_train.shape

(3335, 2, 2, 512)

## What does this mean?
* 303 predictions for 303 images or 3335 predictions for 3335 images when using augmented data set
* 512 bottleneck feature per prediction
* each bottleneck feature has a size of 2x2, just a blob more or less
* bottleneck feature has larger size when we increase size of input images (might be a good idea)
  * 4x4 when using 128x128 as input
  * 8x8 when using 256x256 as input

In [25]:
first_bottleneck_feature = bottleneck_features_train[0,:,:, 0]

In [26]:
first_bottleneck_feature

array([[ 0.        ,  0.        ],
       [ 1.00562787,  0.80484837]], dtype=float32)

## Now we create a new classifier and train it with this output and the labels from ground truth
### Classifier is copied from our first VGG style network

In [27]:
input_shape = bottleneck_features_train.shape[1:]

In [28]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input

# try and vary between .4 and .75
drop_out = 0.50

inputs = Input(shape=input_shape)

x = Flatten()(inputs)

# this is an additional dropout to compensate for the missing one after bottleneck features
x = Dropout(drop_out)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(drop_out)(x)

# softmax activation, 6 categories
predictions = Dense(6, activation='softmax')(x)

In [29]:
classifier_model = Model(input=inputs, output=predictions)
classifier_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 526,086
Trainable params: 526,086
Non-trainable params: 0
_________________________________________________________________


In [30]:
classifier_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
!rm -rf tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=/mnt/c/Users/olive/Development/ml/tf_log
# open http://localhost:6006

## This is a very simple architecture and should train pretty fast
* it overfits by quite a bit

In [32]:
%time history = classifier_model.fit(bottleneck_features_train, y_train, epochs=500, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])
# more epochs might be needed for original data
# %time history = classifier_model.fit(bottleneck_features_train, y_train, epochs=2000, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])

Train on 2668 samples, validate on 667 samples
Epoch 1/500
2668/2668 [==============================] - 0s - loss: 1.9313 - acc: 0.2631 - val_loss: 1.5613 - val_acc: 0.3808
Epoch 2/500
2668/2668 [==============================] - 0s - loss: 1.5772 - acc: 0.3388 - val_loss: 1.4332 - val_acc: 0.4438
Epoch 3/500
2668/2668 [==============================] - 0s - loss: 1.4668 - acc: 0.3842 - val_loss: 1.2979 - val_acc: 0.5367
Epoch 4/500
2668/2668 [==============================] - 0s - loss: 1.3743 - acc: 0.4460 - val_loss: 1.2098 - val_acc: 0.5412
Epoch 5/500
2668/2668 [==============================] - 0s - loss: 1.3106 - acc: 0.4588 - val_loss: 1.1373 - val_acc: 0.6012
Epoch 6/500
2668/2668 [==============================] - 0s - loss: 1.2661 - acc: 0.4925 - val_loss: 1.0884 - val_acc: 0.6087
Epoch 7/500
2668/2668 [==============================] - 0s - loss: 1.2029 - acc: 0.5097 - val_loss: 1.0473 - val_acc: 0.6192
Epoch 8/500
2668/2668 [==============================] - 0s - loss: 1.1

2668/2668 [==============================] - 0s - loss: 0.3178 - acc: 0.8793 - val_loss: 0.4603 - val_acc: 0.8336
Epoch 130/500
2668/2668 [==============================] - 0s - loss: 0.3056 - acc: 0.8894 - val_loss: 0.4625 - val_acc: 0.8426
Epoch 131/500
2668/2668 [==============================] - 0s - loss: 0.3118 - acc: 0.8774 - val_loss: 0.4730 - val_acc: 0.8366
Epoch 132/500
2668/2668 [==============================] - 0s - loss: 0.3319 - acc: 0.8759 - val_loss: 0.4636 - val_acc: 0.8561
Epoch 133/500
2668/2668 [==============================] - 0s - loss: 0.3333 - acc: 0.8722 - val_loss: 0.4463 - val_acc: 0.8561
Epoch 134/500
2668/2668 [==============================] - 0s - loss: 0.2976 - acc: 0.8913 - val_loss: 0.4490 - val_acc: 0.8396
Epoch 135/500
2668/2668 [==============================] - 0s - loss: 0.3217 - acc: 0.8864 - val_loss: 0.4655 - val_acc: 0.8381
Epoch 136/500
2668/2668 [==============================] - 0s - loss: 0.3215 - acc: 0.8812 - val_loss: 0.4544 - val_ac

2668/2668 [==============================] - 0s - loss: 0.2212 - acc: 0.9205 - val_loss: 0.4561 - val_acc: 0.8561
Epoch 258/500
2668/2668 [==============================] - 0s - loss: 0.2230 - acc: 0.9190 - val_loss: 0.4524 - val_acc: 0.8531
Epoch 259/500
2668/2668 [==============================] - 0s - loss: 0.2145 - acc: 0.9190 - val_loss: 0.4659 - val_acc: 0.8486
Epoch 260/500
2668/2668 [==============================] - 0s - loss: 0.1977 - acc: 0.9262 - val_loss: 0.4559 - val_acc: 0.8561
Epoch 261/500
2668/2668 [==============================] - 0s - loss: 0.2191 - acc: 0.9243 - val_loss: 0.4554 - val_acc: 0.8501
Epoch 262/500
2668/2668 [==============================] - 0s - loss: 0.2101 - acc: 0.9239 - val_loss: 0.4529 - val_acc: 0.8546
Epoch 263/500
2668/2668 [==============================] - 0s - loss: 0.2253 - acc: 0.9149 - val_loss: 0.4514 - val_acc: 0.8516
Epoch 264/500
2668/2668 [==============================] - 0s - loss: 0.2021 - acc: 0.9265 - val_loss: 0.4513 - val_ac

2668/2668 [==============================] - 0s - loss: 0.1786 - acc: 0.9299 - val_loss: 0.4907 - val_acc: 0.8516
Epoch 386/500
2668/2668 [==============================] - 0s - loss: 0.2183 - acc: 0.9228 - val_loss: 0.5009 - val_acc: 0.8531
Epoch 387/500
2668/2668 [==============================] - 0s - loss: 0.1934 - acc: 0.9280 - val_loss: 0.4994 - val_acc: 0.8516
Epoch 388/500
2668/2668 [==============================] - 0s - loss: 0.1709 - acc: 0.9423 - val_loss: 0.5013 - val_acc: 0.8456
Epoch 389/500
2668/2668 [==============================] - 0s - loss: 0.1690 - acc: 0.9427 - val_loss: 0.4857 - val_acc: 0.8546
Epoch 390/500
2668/2668 [==============================] - 0s - loss: 0.1753 - acc: 0.9400 - val_loss: 0.4827 - val_acc: 0.8501
Epoch 391/500
2668/2668 [==============================] - 0s - loss: 0.1755 - acc: 0.9389 - val_loss: 0.4846 - val_acc: 0.8516
Epoch 392/500
2668/2668 [==============================] - 0s - loss: 0.1823 - acc: 0.9382 - val_loss: 0.5027 - val_ac

## Issue 1: We have two separate models now
* How do we evaluate?
* How to save model for later prediction use / deployment?

In [33]:
from keras import models

combined_model = models.Sequential()
combined_model.add(vgg_model)
combined_model.add(classifier_model)

In [34]:
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
model_1 (Model)              (None, 6)                 526086    
Total params: 15,240,774
Trainable params: 15,240,774
Non-trainable params: 0
_________________________________________________________________


In [35]:
combined_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
train_loss, train_accuracy = combined_model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

3335/3335 [==============================] - 7s     


(0.11308885638558792, 0.96731634484953077)

In [37]:
test_loss, test_accuracy = combined_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

834/834 [==============================] - 2s     


(0.49626786088486086, 0.85731415182566473)

In [38]:
# complete original non augmented speed limit signs
original_loss, original_accuracy = combined_model.evaluate(original_images, original_labels, batch_size=BATCH_SIZE)
original_loss, original_accuracy

379/379 [==============================] - 1s     


(0.50608633509726508, 0.85488126774891071)

In [39]:
# combined_model.save('vgg16-retrained.hdf5')
combined_model.save('vgg16-augmented-retrained.hdf5')

In [40]:
# !ls -lh vgg16-retrained.hdf5
!ls -lh vgg16-augmented-retrained.hdf5

-rw-rw-r-- 1 ubuntu ubuntu 59M Oct  1 10:00 vgg16-augmented-retrained.hdf5


## Issue 2: Whatever we do, we overfit, much more than 85% on test not possible
* for non augmented data it might even be as low as 70%
* first thing we could try: maybe bottlebeck feature being 2x2 is too small, we could compensate by scaling images up to 128x128 or even 256x256
  * this can indeed bring up test score to 90%
  * however, this will make the model incompatible with the 64x64 input of the other models and make deployment harder, so we keep 64x64
* maybe feature extracting from Imagenet is too different from what we have with speed limit signs? 
* or is the classifier too simply for the complex features?

## Let us try some fine tuning

### First we freeze all but the last convolutional block

In [41]:
len(vgg_model.layers)

19

In [42]:
vgg_model.layers

In [43]:
first_conv_layer = vgg_model.layers[1]

In [44]:
first_conv_layer.trainable

True

In [45]:
# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# so, the general features are kept and we (hopefully) do not have overfitting
non_trainable_layers = vgg_model.layers[:15]

In [46]:
non_trainable_layers

In [47]:
for layer in non_trainable_layers:
    layer.trainable = False

In [48]:
first_conv_layer.trainable

False

### We then tweak the complete model by very slowly re-retraining classifier and final convolutional block
* slow learning prevents us from ruining previous good results
* leave everthing else in place
    * earlier layers hopefully already encode common feaure channels
    * less risk of overfitting
      * earlier layers are more general
      * model has too much capacity for training and is likley to learn each and every detail
    * a little bit faster

#### This may still take quite a while

In [49]:
from keras import optimizers

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate
# make updates very small and non adaptive so we do not ruin previous learnings 
combined_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [50]:
!rm -r tf_log

In [51]:
%time combined_model.fit(X_train, y_train, epochs=150, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])
# non augmented data is cheap to retrain, so we can try a few more epochs
# %time combined_model.fit(X_train, y_train, epochs=1000, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])

Train on 2668 samples, validate on 667 samples
Epoch 1/150
2668/2668 [==============================] - 9s - loss: 0.1467 - acc: 0.9509 - val_loss: 0.5544 - val_acc: 0.8471
Epoch 2/150
2668/2668 [==============================] - 6s - loss: 0.1582 - acc: 0.9460 - val_loss: 0.5605 - val_acc: 0.8561
Epoch 3/150
2668/2668 [==============================] - 6s - loss: 0.1675 - acc: 0.9449 - val_loss: 0.5098 - val_acc: 0.8576
Epoch 4/150
2668/2668 [==============================] - 6s - loss: 0.1700 - acc: 0.9389 - val_loss: 0.5663 - val_acc: 0.8426
Epoch 5/150
2668/2668 [==============================] - 6s - loss: 0.1463 - acc: 0.9483 - val_loss: 0.5123 - val_acc: 0.8591
Epoch 6/150
2668/2668 [==============================] - 6s - loss: 0.1522 - acc: 0.9528 - val_loss: 0.4983 - val_acc: 0.8546
Epoch 7/150
2668/2668 [==============================] - 6s - loss: 0.1399 - acc: 0.9520 - val_loss: 0.5469 - val_acc: 0.8516
Epoch 8/150
2668/2668 [==============================] - 6s - loss: 0.1

2668/2668 [==============================] - 6s - loss: 0.0218 - acc: 0.9936 - val_loss: 0.4493 - val_acc: 0.8981
Epoch 130/150
2668/2668 [==============================] - 6s - loss: 0.0248 - acc: 0.9929 - val_loss: 0.4862 - val_acc: 0.8951
Epoch 131/150
2668/2668 [==============================] - 6s - loss: 0.0196 - acc: 0.9929 - val_loss: 0.4992 - val_acc: 0.9025
Epoch 132/150
2668/2668 [==============================] - 6s - loss: 0.0159 - acc: 0.9936 - val_loss: 0.5076 - val_acc: 0.9025
Epoch 133/150
2668/2668 [==============================] - 6s - loss: 0.0255 - acc: 0.9925 - val_loss: 0.5199 - val_acc: 0.9040
Epoch 134/150
2668/2668 [==============================] - 6s - loss: 0.0128 - acc: 0.9959 - val_loss: 0.5309 - val_acc: 0.8951
Epoch 135/150
2668/2668 [==============================] - 6s - loss: 0.0163 - acc: 0.9970 - val_loss: 0.5430 - val_acc: 0.8966
Epoch 136/150
2668/2668 [==============================] - 6s - loss: 0.0151 - acc: 0.9948 - val_loss: 0.5435 - val_ac

## 90% for validation is quite a bit of improvement, might even increase when we train for a bit longer

## Metrics for Augmented Data

### Accuracy
![Accuracy Fine Tuning](https://djcordhose.github.io/ai/img/tensorboard/cnn-acc-fine-tuning.png)
### Validation Accuracy
![Validation Accuracy Fine Tuning](https://djcordhose.github.io/ai/img/tensorboard/cnn-val-acc-fine-tuning.png)

In [52]:
train_loss, train_accuracy = combined_model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

3335/3335 [==============================] - 6s     


(0.10222922369810358, 0.98020990570445821)

In [53]:
test_loss, test_accuracy = combined_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

834/834 [==============================] - 1s     


(0.42116924984563742, 0.89928059569365681)

In [54]:
# complete original non augmented speed limit signs
original_loss, original_accuracy = combined_model.evaluate(original_images, original_labels, batch_size=BATCH_SIZE)
original_loss, original_accuracy

379/379 [==============================] - 0s     


(0.45832276033694636, 0.90501321683143876)

In [55]:
combined_model.save('vgg16-augmented-retrained-fine-tuned.hdf5')
# combined_model.save('vgg16-retrained-fine-tuned.hdf5')

In [56]:
# !ls -lh vgg16-retrained-fine-tuned.hdf5
!ls -lh vgg16-augmented-retrained-fine-tuned.hdf5

-rw-rw-r-- 1 ubuntu ubuntu 88M Oct  1 10:22 vgg16-augmented-retrained-fine-tuned.hdf5
